# Flattened 1-D Multi-Input Channel Convolution

This notebook uses the fiber-tree emulator to display the behaviour of flattening multiple input channel 1-D into a simple 1-D convolution of a single channel. 

Note this notebook relies on the invariant that the data is dense. Therefore, because the data is assumed to be dense we can use the position-based operators on the premise that for dense data the position and coordinate are the same.

First, include some libraries

In [ ]:
# Run boilerplate code to set up environment

%run ../prelude.py --style=uncompressed --animation=movie

## Convolution Inputs Selection

Using sliders to select the shapes of the weights and input activations

In [ ]:
# Initial values

C = 2
S = 3
W = 8
Q = W-S+1

density = [1.0]
seed = 10

enable_log = False

def set_params(rank_C, rank_S, rank_W, rand_seed, log):
    global C
    global S
    global W
    global Q
    global density
    global seed
    global enable_log
    
    C = rank_C
    S = rank_S
    W = rank_W
    Q = W-S+1
    
    seed = rand_seed

    enable_log = (log == 'enable')

def log(*args):
    if enable_log:
        print(*args)

controls = interactive(set_params,
                       rank_C=widgets.IntSlider(min=1, max=4, step=1, value=C),
                       rank_S=widgets.IntSlider(min=2, max=9, step=1, value=S),
                       rank_W=widgets.IntSlider(min=2, max=12, step=1, value=W),
                       rand_seed=widgets.IntSlider(min=0, max=100, step=1, value=seed),
                       log=['disable', 'enable'])


display(controls)

## Create Input Tensors

Given shapes selected above create and display the filter weights (**f**) and input activations (**i**) and a reference output (**o_verify**)

In [ ]:
i_raw = []
f_raw = []

for c in range(C):
    i_raw.append([random.randint(1, 6) for i in range(W)])
    f_raw.append([random.randint(1, 6) for i in range(S)])

o_verify_raw = [0 for i in range(Q)]

for q in range(Q):
    for c in range(C):
        for s in range(S):
            w = q+s
            o_verify_raw[q] += i_raw[c][w] * f_raw[c][s]

i = Tensor.fromUncompressed(["C", "W"], i_raw)
f = Tensor.fromUncompressed(["C", "S"], f_raw)
o_verify = Tensor.fromUncompressed(["Q"], o_verify_raw)

i.setName("I").setColor("blue")
f.setName("F").setColor("green")
o_verify.setName("O")

print("Input activations")
displayTensor(i)
print("Filter Weights")
displayTensor(f)
print("Output activations (expected)")
displayTensor(o_verify)


## Multiple Input Channel Convolution

Process the convolution of the multi-channel input activation and filter weight tensors.

In [ ]:
o = Tensor(rank_ids=["Q"]).setName("O")

# Hack to fill in all the entries in o
o_q = o.getRoot()
o_q << Fiber(coords=range(Q), initial=1)

print("Convolution")

output_shape = Fiber(coords=range(Q), initial=1)

canvas = createCanvas(f, i, o)

for q in range(Q):
    log(f"Processing output: ({q}, ({o[q]}))")
    for c in range(C):
        for w in range(W):
            s = w - q
            if s < 0 or s >= S: continue
            log(f"Processing input: ({c}, {w}, {i[c][w]})")
            log(f"  Processing filter weight ({c}, {s}, {f[c][s]})")
            o[q] += f[c][s] * i[c][w]

            canvas.addActivity((), [(c, w,) for w in range(q, q+S)], (), worker="W")
            canvas.addFrame((c, s,), (c, w,), (q,))

displayTensor(o)
displayCanvas(canvas)

print("Input Activations - before")
assert o == o_verify

## Flatten Inputs and Filters

Flatten the input activations and filter weights into a single input channel.

In [ ]:
print("Filter Weights - before")
displayTensor(f)

f_flat = f.swapRanks().flattenRanks()
f_flat.getRoot().updateCoords(lambda pos, c, p: (c[0]*C)+c[1])
f_flat.setName("F_flat")

print("Filter Weights - after")
displayTensor(f_flat)


print("Input Activations - before")
displayTensor(i)

i_flat = i.swapRanks().flattenRanks()
i_flat.getRoot().updateCoords(lambda pos, c, p: (c[0]*C)+c[1])
i_flat.setName("I_flat")

print("Input Activations - after")
displayTensor(i_flat)




## Flattened convolution

Process a convolution on the flattened input activations and filter weights. Note that the window now slides by ```C``` the number of inputs channels.

Note, this processing pattern is the same as used by Eyeriss for processing multiple input channels at once in a single PE.

In [ ]:
o = Tensor(rank_ids=["Q"]).setName("O")

# Hack to fill in all the entries in o
o_q = o.getRoot()
o_q << Fiber(coords=range(Q), initial=1)

print("Convolution")

output_shape = Fiber(coords=range(Q), initial=1)

canvas = createCanvas(f_flat, i_flat, o)

for q in range(Q):
    log(f"Processing output: ({q}, ({o[q]}))")
    for w in range(C*W):
        s = w - C*q
        if s < 0 or s >= C*S: continue
        log(f"Processing input: ({w}, {i_flat[w]})")
        log(f"  Processing filter weight ({s}, {f_flat[s]})")
        o[q] += f_flat[s] * i_flat[w]

        canvas.addActivity((), [(w,) for w in range(C*q, C*q+C*S)], (), worker="W")
        canvas.addFrame((s,), (w,), (q,))

displayTensor(o)
displayCanvas(canvas)

assert o == o_verify

## Testing area

For running alternative algorithms